In [21]:
import streamlit as st
import pickle
import base64
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from glob import glob
import pandas as pd
import docx2txt
from win32com import client
import os
from docx import Document 
import PyPDF2
import textract
import spacy
import json
import re
from nltk.tokenize import TweetTokenizer
import string
from collections import Counter
import string
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import sweetviz
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,roc_auc_score,confusion_matrix

In [8]:
# Background
# Set page config
st.set_page_config(page_title="My Streamlit App", page_icon=":smiley:", layout="wide")

# Set background image
def add_bg_from_local(image_file):
    with open(image_file, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
    st.markdown(
    f"""
    <style>
    .stApp {{
        background-image: url(data:image/{"png"};base64,{encoded_string.decode()});
        background-size: cover
    }}
    </style>
    """,
    unsafe_allow_html=True
    )
#add_bg_from_local("D:\dummy\strimlate\strimlate\white.webp")
    
# Add title with CSS styles
st.markdown("""
<style>
h1 {
    text-align: center;
    color: #333333;
}
</style>
""", unsafe_allow_html=True)

st.title("GROUP 2 Project")
# load the pre-trained LSTM model
loaded_model=pickle.load(open(r'modelNLP1.pkl','rb'))

# Assigning path based on the category
files1 = glob("D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\classified_Resumes\\Peoplesoft resumes\\*")
files2 = glob("D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\classified_Resumes\\React JS Developer\\*")
files3 = glob("D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\classified_Resumes\\SQL Developer Lightning insight\\*")
files4 = glob("D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\classified_Resumes\\workday resumes\\*")


In [9]:
# funtion for reading doc files
def docReader(doc_file_name): 
    ## 1) Initiate an object that interfaces to Word
    word = client.Dispatch("Word.Application")
    word.Visible = False 
    
    ## 2) Open the Word document to read in
    _ = word.Documents.Open(doc_file_name)

    ## 3) Extract the paragraphs and close the connections
    doc = word.ActiveDocument
    paras = doc.Range().text    
    doc.Close()
    word.Quit()
    return paras    

In [10]:
def convef1():

   # Importing the data present in first file (People soft)
        data1 = []
        for i in range(len(files1)):
            if files1[i].endswith('docx'):
                x = docx2txt.process(files1[i]);
                data1.append(x)
            if files1[i].endswith('doc'):
                y = docReader(files1[i]);
                data1.append(y)
                [a for a in y.replace('\x07', '\r').split('\r') if a]
            z = None  # define a default value for z
            if files1[i].endswith('pdf'):
                z = PyPDF2.PdfReader(files1[i]);
                z1 = ''
                for j in range(len(z.pages)):
                    m = z.pages[j].extract_text()
                    z1 = z1 + m
                data1.append(z1)


        data1 = pd.DataFrame(data=data1,columns=['data']) # converting to DataFrame
        data1['category'] = 'Peoplesoft' # Creating column with column name category and assigning "peoplesoft" to every cell
        # Adding Name column and assinging name
        name1 = []
        for i in range(len(files1)):
            tem = files1[i].split('\\');
            name1.append(tem[-1])
            names1 = []
        for i in range(len(name1)):
            d = name1[i].split('.')
            names1.append(d[0])
        names1 = pd.DataFrame(data = names1,columns=["Name"])
        data1 = pd.concat([data1,names1],axis=1)
        return data1

        
def convef2():
    # Importing the data present in second file (React JS Developer)
        data2 = []
        for i in range(len(files2)):
            if files2[i].endswith('docx'):
                x = docx2txt.process(files2[i]);
                data2.append(x)
            if files2[i].endswith('doc'):
                y = docReader(files2[i]);
                data2.append(y)
                [a for a in y.replace('\x07', '\r').split('\r') if a]
            if files2[i].endswith('pdf'):
                z = PyPDF2.PdfReader(files2[i]);
                z1 = ''
                for j in range(len(z.pages)):
                    m = z.pages[j].extract_text()
                    z1 = z1 + m
                data2.append(z1)
        data2 = pd.DataFrame(data=data2,columns=['data'])
        data2['category'] = 'React JS Developer' # Creating column with column name category and assigning "React JS Developer" to every cell
        # Adding Name column and assinging name
        name2 = []
        for i in range(len(files2)):
            tem = files2[i].split('\\');
            name2.append(tem[-1])
        names2 = []
        for i in range(len(name2)):
            d = name2[i].split('.')
            names2.append(d[0])
        names2 = pd.DataFrame(data = names2,columns=["Name"])
        data2 = pd.concat([data2,names2],axis=1)
        return data2

def convef3():
    # Importing the data present in third file (SQL Developer)
    data3 = []
    for i in range(len(files3)):
        if files3[i].endswith('docx'):
            x = docx2txt.process(files3[i]);
            data3.append(x)
        if files3[i].endswith('doc'):
            y = docReader(files3[i]);
            data3.append(y)
            [a for a in y.replace('\x07', '\r').split('\r') if a]
        if files3[i].endswith('pdf'):
            z = PyPDF2.PdfReader(files3[i]);
            z1 = ''
            for j in range(len(z.pages)):
                m = z.pages[j].extract_text()
                z1 = z1 + m
            data3.append(z1)
    data3 = pd.DataFrame(data=data3,columns=['data'])
    data3['category'] = 'SQL Developer' # Creating column with column name category and assigning "SQL Developer" to every cell 
    # Adding Name column and assinging name
    name3 = []
    for i in range(len(files3)):
        tem = files3[i].split('\\');
        name3.append(tem[-1])
    names3 = []
    for i in range(len(name3)):
        d = name3[i].split('.')
        names3.append(d[0])
    names3 = pd.DataFrame(data = names3,columns=["Name"])
    data3 = pd.concat([data3,names3],axis=1)
    return data3

def convef4():
    # Importing the data present in 4th file (workday)
    data4 = []
    for i in range(len(files4)):
        if files4[i].endswith('docx'):
            x = docx2txt.process(files4[i]);
            data4.append(x)
        if files4[i].endswith('doc'):
            y = docReader(files4[i]);
            data4.append(y)
            [a for a in y.replace('\x07', '\r').split('\r') if a]
        if files4[i].endswith('pdf'):
            z = PyPDF2.PdfReader(files4[i]);
            z1 = ''
            for j in range(len(z.pages)):
                m = z.pages[j].extract_text()
                z1 = z1 + m
            data4.append(z1)
    data4 = pd.DataFrame(data=data4,columns=['data'])
    data4['category'] = 'workday' # Creating column with column name category and assigning "workday" to every cell 
    # Adding Name column and assinging name
    name4 = []
    for i in range(len(files4)):
        tem = files4[i].split('\\');
        name4.append(tem[-1])
    names4 = []
    for i in range(len(name4)):
        d = name4[i].split('.')
        names4.append(d[0])
    names4 = pd.DataFrame(data = names4,columns=["Name"])
    data4 = pd.concat([data4,names4],axis=1)
    return data4


In [11]:
def addata():
    data1=convef1()
    data2=convef2()
    data3=convef3()
    data4=convef4()
    df = pd.DataFrame()
    
    lst = [data1,data2,data3,data4]
    for subDF in lst:
        df = pd.concat([df, subDF],ignore_index=True)
    return df

In [ ]:
def skile():
    df=addata()
    test = spacy.load('en_core_web_sm')
    Skills = []
    for i in range(len(df.data)):
        ts = test(" ".join(df.data[i].split('\n'))) # we have splitted our data with '\n' and rejoined with space. 
        tt = []
        for ent in ts.ents:
            if ent.label_.upper() == 'ORG':
                tt.append(ent.text)
        Skills.append(tt) # appending all skills to the list skills
    df['skills']=0  # creating new columns skills and assiging 0 to every column
    for i in range(len(df.skills)):
            df.skills[i] = Skills[i]
    return df

In [13]:
def eda():
    df=skile()
    for i in range(len(df.skills)):
        lower_words=[Text.lower() for Text in df.skills[i]]
    df.skills[i] = lower_words
    for i in range(len(df.skills)):
        ab =[]
    for j in range(len(df.skills[i])):
        jk = re.split(r'[,(\n\t:]', df.skills[i][j]) # splitting the objects using ,,(,\n,\t,:
        ab = jk + ab
        df.skills[i] = ab
    # finding the duplicat values
    return df

In [14]:
def duplica():
    df=eda()
    # finding the duplicat values
    for i in range(len(df.skills)):
        numbers = df.skills[i]
        counts = dict(Counter(numbers))
        duplicates = {key:value for key, value in counts.items() if value > 1}
        st.write(duplicates)
        df= duplica()
    for i in range(len(df.skills)):
        lm = set(df.skills[i])
        df.skills[i] = list(lm)
    return df

In [15]:
def remoo():
    df=duplica()
    
        # Removing the unwanted data like '',' ','s','cs' which contains length upto 2
    for i in range(len(df.skills)):
        er = []
        for j in range(len(df.skills[i])):
            if (len(df.skills[i][j]) >= 3) :
                ab = df.skills[i][j]
                er = er + [ab]
        df.skills[i] = er
    # Removing all punctuation

    for i in range(len(df.skills)):
        for j in range(len(df.skills[i])):
            df.skills[i][j] = df.skills[i][j].translate(str.maketrans('','',string.punctuation))
    for i in range(len(df.skills)):
        for j in range(len(df.skills[i])):
            df.skills[i][j] = ''.join([i for i in df.skills[i][j] if not i.isdigit()])
    # Removing all spaces 
   
    for i in range(len(df.skills)):
        for j in range(len(df.skills[i])):
            df.skills[i][j] = word_tokenize(df.skills[i][j])
            df.skills[i][j] =  ' '.join(df.skills[i][j])
    # Removing the unwanted data like '',' ','s','cs' which contains length upto 2
    
    for i in range(len(df.skills)):
        er = []
        for j in range(len(df.skills[i])):
            if (len(df.skills[i][j]) >= 3) :
                ab = df.skills[i][j]
                er = er + [ab]
        df.skills[i] = er
    
    
    
    return df

In [16]:
def coppy():
   df=remoo()
   df1 = df.copy(deep=True) # it will take copy of df 
   for i in range(len(df1.skills)):
        df1.skills[i] = " ".join(df1.skills[i]) # converting list into string
   for i in range(len(df1.skills)):
    df1.skills[i] = word_tokenize(df1.skills[i]) # tokenization
    nltk.download('stopwords') # importing stop words

    my_stop_words = stopwords.words('english')
    my_stop_words.append(' ')
    my_stop_words.append('&') # adding reqiued stop words
    # removing stop words
    for i in range(len(df1.skills)):    
        df1.skills[i] = [word for word in df1.skills[i] if not word in my_stop_words ]
    # joining the words into single document (removing the tokenization)
    for i in range(len(df1.skills)):
        df1.skills[i] =  ' '.join(df1.skills[i])
    # Lemmatization
    Lemmatizer = WordNetLemmatizer()
    for i in range(len(df1.skills)):
        lemmas = []
        for token in df1.skills[i].split():
            lemmas.append(Lemmatizer.lemmatize(token))
        df1.skills[i] = lemmas
    # joining the words into single document (removing the tokenization)
    for i in range(len(df1.skills)):
        df1.skills[i] =  ' '.join(df1.skills[i])
    return df1

In [17]:
def text():
    df1=coppy()
    df=remoo()
    for i in range(len(df1.skills)):
        df2 = [skills.strip() for skills in df1.skills] # adding all rows to one column
        text = " ".join(df2)
        text_tokens = word_tokenize(text)
        text1 = pd.DataFrame(data=text_tokens,columns=['text'])
        text1['y'] = 1
    df3 = []
    for i in range(len(df.skills)):
        df3 = df3 + df.skills[i]
    # Lemmatization
    Lemmatizer = WordNetLemmatizer()
    lemmas = []
    for token in df3:
        lemmas.append(Lemmatizer.lemmatize(token))
    df3 = pd.DataFrame(data=df3,columns = ['data'])
    df3['y'] = 1
    return df3

In [18]:
def extaraction():
    df1=coppy()
    x = df1['skills']
    y = df1['category']

    word_vectorizer = TfidfVectorizer(
        sublinear_tf=True)
    word_vectorizer.fit(x)
    x = word_vectorizer.transform(x)
    LE = LabelEncoder()
    y = LE.fit_transform(y)
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=30, test_size=0.20, shuffle = True, stratify=y)
    return x_train, x_test, y_train, y_test

In [19]:
def model():
    x_train, x_test, y_train, y_test=extaraction()
    Knn = KNeighborsClassifier(n_neighbors=5,p=2)
    Knn.fit(x_train, y_train)
    y_pred_train = Knn.predict(x_train)
    y_pred_test = Knn.predict(x_test)
    
    a21 = accuracy_score(y_train, y_pred_train)
    a22 =accuracy_score(y_test, y_pred_test)
    r21 = recall_score(y_train, y_pred_train,average = 'macro')
    r22 = recall_score(y_test, y_pred_test,average = 'macro')
    p21 = precision_score(y_train, y_pred_train,average = 'macro')
    p22 = precision_score(y_test, y_pred_test,average = 'macro')
    f21 = f1_score(y_train, y_pred_train,average = 'macro')
    f22 = f1_score(y_test, y_pred_test,average = 'macro')
    m2 = 'KNN'
    model = {'model':[m2],
         "train accuracy":[a21],'test accuracy':[a22],
         'train recall':[r21],"test recall":[r22],
         'train precision':[p21],"test precision":[p22],
         'tain f1_score':[p21],'test f1_score':[f22]}
    model = pd.DataFrame(model)
    return model

In [ ]:
def main():
    #giving title
    st.title('Forcasting future data web app')

    #getting input variable from users
    n_future=st.text_input('Number of future data')

    diagnosis=''
    
    
    
    df=model()
    #creating button for prediction
    st.write(df)
if __name__ == '__main__':
    main()

C:\Users\Vinay Sai\AppData\Local\Temp\ipykernel_27080\3937734363.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.skills[i] = skills[i]
C:\Users\Vinay Sai\AppData\Local\Temp\ipykernel_27080\3937734363.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.skills[i] = skills[i]
C:\Users\Vinay Sai\AppData\Local\Temp\ipykernel_27080\3937734363.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.skills[i] = skills[i]
C:\Users\V